In [ ]:
# this notebook is running on google colab

In [58]:
using Pkg
Pkg.status()

      Status `~/.julia/environments/v1.7/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.1
  [052768ef] CUDA v3.9.0
  [7073ff75] IJulia v1.23.3
  [b8d414a4] PaddleChainRules v0.1.0 `https://github.com/songjhaha/PaddleChainRules.jl.git#main`
  [438e738f] PyCall v1.93.1
  [e88e6eb3] Zygote v0.6.38


In [59]:
using PyCall
# install paddle-gpu
run(`$(PyCall.pyprogramname) -m pip install paddlepaddle-gpu`)

Process(`/usr/bin/python3 -m pip install paddlepaddle-gpu`, ProcessExited(0))

In [60]:
# cpu information
run(`lscpu`)

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               63
Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:            0
CPU MHz:             2299.998
BogoMIPS:            4599.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            46080K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs 

Process(`lscpu`, ProcessExited(0))

In [61]:
# gpu information
run(`nvidia-smi`)

Sat Apr 16 03:35:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    72W / 149W |    949MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Process(`nvidia-smi`, ProcessExited(0))

In [62]:
using PaddleChainRules.Paddle: paddle, PaddleModuleWrapper, PaddleStatelessModule, ispysetup
using BenchmarkTools
using Zygote
using CUDA

if !ispysetup[]
    return
end

function forward_pass!(s, indim, outdim, hiddendim, dummyinput, batchsize, device; samples=100, evals=10)
    if device == "cuda"
        paddle.device.set_device("gpu:0")
    else
        paddle.device.set_device("cpu")
    end
    ml = paddle.nn.Sequential(paddle.nn.Linear(indim, hiddendim), paddle.nn.ReLU(), paddle.nn.Linear(hiddendim, outdim))
    mlfun = PaddleStatelessModule(ml)
    pyparams = deepcopy(ml.parameters())
    mlfun2 = x->mlfun(pyparams, x)

    mlwrap = PaddleModuleWrapper(deepcopy(ml))

    
    inshape = size(dummyinput)

    pyinputs = paddle.randn((batchsize, inshape...))
    jlinputs = randn(Float32, reverse(inshape)..., batchsize)
    if device == "cuda"
        jlinputs = CUDA.cu(jlinputs)
    end


    ## Paddle
    b_paddle = @benchmarkable($ml(x), setup=(x=$pyinputs), samples=samples, evals=evals)

    ## FuncPaddle
    b_funcpaddle = @benchmarkable($mlfun2(x), setup=(x=$pyinputs), samples=samples, evals=evals)

    ## PaddleModuleWrapper
    b_jl = @benchmarkable($mlwrap(x), setup=(x=$jlinputs), samples=samples, evals=evals)
    
    
    s["paddle"] = b_paddle
    s["funcpaddle"] = b_funcpaddle
    s["jl"] = b_jl
end

function backward_pass!(s, indim, outdim, hiddendim, dummyinput, batchsize, device; samples=100, evals=10)
    if device == "cuda"
        paddle.device.set_device("gpu:0")
    else
        paddle.device.set_device("cpu")
    end
    ml = paddle.nn.Sequential(paddle.nn.Linear(indim, hiddendim), paddle.nn.ReLU(), paddle.nn.Linear(hiddendim, outdim))
    mlfun = PaddleStatelessModule(ml)
    pyparams = deepcopy(ml.parameters())

    mlwrap = PaddleModuleWrapper(deepcopy(ml))


    inshape = size(dummyinput)

    pyinputs = paddle.randn((batchsize, inshape...))
    jlinputs = randn(Float32, reverse(inshape)..., batchsize)
    if device == "cuda"
        jlinputs = CUDA.cu(jlinputs)
    end

    ## Paddle
    paddlegrad = (m, x)->paddle.grad(paddle.sum(m(x)), m.parameters())
    b_paddle = @benchmarkable($paddlegrad(m, x), setup=(m=$ml; x=$pyinputs), samples=samples, evals=evals)

    ## Funcpaddle
    funcgrad = (params, x)->paddle.grad(paddle.sum(mlfun(params, x)), params)
    b_funcpaddle = @benchmarkable($funcgrad(params, x), setup=(params=$pyparams; x=$pyinputs), samples=samples, evals=evals)

    ## PaddleModuleWrapper
    zygotegrad = (m, x) -> Zygote.gradient(m->sum(m(x)), m)
    b_jl = @benchmarkable($zygotegrad(m, x), setup=(m=$mlwrap; x=$jlinputs), samples=samples, evals=evals)
    
    s["paddle"] = b_paddle
    s["funcpaddle"] = b_funcpaddle
    s["jl"] = b_jl
end

backward_pass! (generic function with 1 method)

In [64]:
batchsizes = [1, 8, 16, 32]
indim = 8
outdim = 4
hiddendim = 64

suite = BenchmarkGroup()

0-element BenchmarkTools.BenchmarkGroup:
  tags: []

In [65]:
for device in ["cpu", "cuda"]
    ss = suite["$device"] = BenchmarkGroup()
    for batchsize in batchsizes
        s = ss["bs=$batchsize"] = BenchmarkGroup()
        # Forward pass
        s["forward"] = BenchmarkGroup()
        forward_pass!(s["forward"], indim, outdim, hiddendim, randn(Float32, indim), batchsize, device)

        # Gradient pass
        s["backward"] = BenchmarkGroup()
        backward_pass!(s["backward"], indim, outdim, hiddendim, randn(Float32, indim), batchsize, device)
    end
end

In [66]:
# take a long time
tune!(suite)

2-element BenchmarkTools.BenchmarkGroup:
  tags: []
  "cpu" => 4-element BenchmarkTools.BenchmarkGroup:
	  tags: []
	  "bs=16" => 2-element BenchmarkTools.BenchmarkGroup:
		  tags: []
		  "forward" => 3-element BenchmarkTools.BenchmarkGroup:
			  tags: []
			  "funcpaddle" => Benchmark(evals=1, seconds=5.0, samples=100)
			  "jl" => Benchmark(evals=1, seconds=5.0, samples=100)
			  "paddle" => Benchmark(evals=1, seconds=5.0, samples=100)
		  "backward" => 3-element BenchmarkTools.BenchmarkGroup:
			  tags: []
			  "funcpaddle" => Benchmark(evals=1, seconds=5.0, samples=100)
			  "jl" => Benchmark(evals=1, seconds=5.0, samples=100)
			  "paddle" => Benchmark(evals=1, seconds=5.0, samples=100)
	  "bs=32" => 2-element BenchmarkTools.BenchmarkGroup:
		  tags: []
		  "forward" => 3-element BenchmarkTools.BenchmarkGroup:
			  tags: []
			  "funcpaddle" => Benchmark(evals=1, seconds=5.0, samples=100)
			  "jl" => Benchmark(evals=1, seconds=5.0, samples=100)
			  "paddle" => Benchmark(evals=1,

In [67]:
results = run(suite, verbose = false)

2-element BenchmarkTools.BenchmarkGroup:
  tags: []
  "cpu" => 4-element BenchmarkTools.BenchmarkGroup:
	  tags: []
	  "bs=16" => 2-element BenchmarkTools.BenchmarkGroup:
		  tags: []
		  "forward" => 3-element BenchmarkTools.BenchmarkGroup:
			  tags: []
			  "funcpaddle" => Trial(310.671 μs)
			  "jl" => Trial(752.794 μs)
			  "paddle" => Trial(270.323 μs)
		  "backward" => 3-element BenchmarkTools.BenchmarkGroup:
			  tags: []
			  "funcpaddle" => Trial(853.963 μs)
			  "jl" => Trial(1.749 ms)
			  "paddle" => Trial(864.520 μs)
	  "bs=32" => 2-element BenchmarkTools.BenchmarkGroup:
		  tags: []
		  "forward" => 3-element BenchmarkTools.BenchmarkGroup:
			  tags: []
			  "funcpaddle" => Trial(319.946 μs)
			  "jl" => Trial(763.199 μs)
			  "paddle" => Trial(270.306 μs)
		  "backward" => 3-element BenchmarkTools.BenchmarkGroup:
			  tags: []
			  "funcpaddle" => Trial(870.366 μs)
			  "jl" => Trial(1.778 ms)
			  "paddle" => Trial(918.160 μs)
	  "bs=8" => 2-element BenchmarkTools.Benc